# Assignment: Determinant and Inverse

---

<b><div style="text-align: right">[TOTAL POINTS: 15]</div></b>

This assignment is based on the use of determinants and inverses in regression. We will attempt to predict the price of a building based on the features of that particular property.

## Dataset Description

**Source**: The dataset used here is from a Kaggle challenge - [House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data). It is a housing dataset that relates the price of a property based on the features of the property. The version of the dataset that you will use in this assignment does not contain all the features that exist in the original dataset, and some new features have been added too. The dataset presents the data for houses sold in the year 2019. 

**Number of Instances**: 1000

**Number of Attributes**: 10

---

The features used for the prediction are as follows:

1. `Building_design` : Type of house in terms of its design
    - 1 - A building with one story
    - 2 - A building with 2 storues
    - 1.5 - A building with 1 and half stories
2. `Area`: Area of house in square feet
3. `Width`: Width of plot in feet
4. `Length`: Length of house in feet
5. `Circumgerence`: Circumference of the base of the house in feet
6. `Year_built`: The year that the house was built in
7. `Age`: Age of the house when sold
8. `No_bedrooms`: Number of bedrooms in the house
9. `Quality`: Quality of the house, ranging from 1 (low) to 10 (high)

The target of the dataset is the `Price` of the building at the time of sale based on the above 9 attributes.


In [1]:
# Import libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from numpy.linalg import LinAlgError, matrix_rank as rank

! wget -qO linalg.py 'https://storage.googleapis.com/codehub-data/5-A-2-5-linalg.py'
# custom library
from linalg import det, inv

**Note:** The functions `det` and `inv` imported above from `linalg` are different from the NumPy implementations of determinant and inverse.  They make sure that problems due to floating point precision do not change the result of the determinant and inverse of a matrix. Use them in place of `np.linalg.det` and `np.linalg.inv` respectively.

In [43]:
# Read the housing data file
df = pd.read_csv('https://storage.googleapis.com/codehub-data/5-A-2-5-housing_data.csv')
df.head() 

,Building_design,Area,Width,Length,Circumference,Year_built,Age,No_bedroom,Quality,Price
0,2.0,8450,65,130,390,2012,7,3,7,208500
1,1.0,9600,80,120,400,1985,34,3,6,181500
2,2.0,11250,68,165,466,2010,9,3,7,223500
3,2.0,9550,60,159,438,1924,95,3,7,140000
4,1.0,10084,75,134,418,2013,6,3,8,307000


In [44]:
# All the columns except for the last one is assigned to input data matrix X
X = df.values[:, :-1]

# The last column that is Price is assigned to the variable y
y = df.values[:, -1]

In [45]:
# print first five rows of X
X[:5]

array([[2.0000e+00, 8.4500e+03, 6.5000e+01, 1.3000e+02, 3.9000e+02,
        2.0120e+03, 7.0000e+00, 3.0000e+00, 7.0000e+00],
       [1.0000e+00, 9.6000e+03, 8.0000e+01, 1.2000e+02, 4.0000e+02,
        1.9850e+03, 3.4000e+01, 3.0000e+00, 6.0000e+00],
       [2.0000e+00, 1.1250e+04, 6.8000e+01, 1.6500e+02, 4.6600e+02,
        2.0100e+03, 9.0000e+00, 3.0000e+00, 7.0000e+00],
       [2.0000e+00, 9.5500e+03, 6.0000e+01, 1.5900e+02, 4.3800e+02,
        1.9240e+03, 9.5000e+01, 3.0000e+00, 7.0000e+00],
       [1.0000e+00, 1.0084e+04, 7.5000e+01, 1.3400e+02, 4.1800e+02,
        2.0130e+03, 6.0000e+00, 3.0000e+00, 8.0000e+00]])

The **target** of the assignment is to obtain a vector $w$ such that $Xw=\hat{y}$ and $\hat{y}$ approximates $y$ as closely as possible. 

In [46]:
print(f'Shape of X = {X.shape}')

Shape of X = (1000, 9)


Since, the dataset contains $10000$ datapoints and only $10$ rows (including Price - our target variable), data matrix $X$ is a rectangular one with shape $1000 \times 9 $ as seen above. Next, we need to append a vector of length $1000$ of all ones to data matrix. This allows us to introduce a constant bias to our model.

In [47]:
# Append (1000,1) ones to X
X = np.concatenate([np.ones((1000, 1)), X], axis=1)

In [48]:
# Now print shape of X
print(X.shape)

(1000, 10)


So, the `ones` has been appended to X and the new shape is $(1000, 1)$.

In [49]:
# Print 5 rows of X. Here each element of X is a list with 10 numbers and 1 added to each list in the beginning
print(X[:5])

[[1.0000e+00 2.0000e+00 8.4500e+03 6.5000e+01 1.3000e+02 3.9000e+02
  2.0120e+03 7.0000e+00 3.0000e+00 7.0000e+00]
 [1.0000e+00 1.0000e+00 9.6000e+03 8.0000e+01 1.2000e+02 4.0000e+02
  1.9850e+03 3.4000e+01 3.0000e+00 6.0000e+00]
 [1.0000e+00 2.0000e+00 1.1250e+04 6.8000e+01 1.6500e+02 4.6600e+02
  2.0100e+03 9.0000e+00 3.0000e+00 7.0000e+00]
 [1.0000e+00 2.0000e+00 9.5500e+03 6.0000e+01 1.5900e+02 4.3800e+02
  1.9240e+03 9.5000e+01 3.0000e+00 7.0000e+00]
 [1.0000e+00 1.0000e+00 1.0084e+04 7.5000e+01 1.3400e+02 4.1800e+02
  2.0130e+03 6.0000e+00 3.0000e+00 8.0000e+00]]


Run the following two scripts and observe their output

In [50]:
# Try to calculate determinant of matrix X
try:
    det(X)  
except LinAlgError as err:
    print("Error : " + str(err))

Error : Last 2 dimensions of the array must be square


In [51]:
try: 
    w = inv(X) @ y
except LinAlgError as err:
    print("Error : " + str(err))

Error : Last 2 dimensions of the array must be square


$X$ is not a square matrix so, its determinat is not defines as determinant is defined for square matrices only. Similarly, inverse (not right, left, or pseudo defined for rectangluar matrix too) of a matrix is defined on a square matrix only. Hence above, two errors have appeared.

Following the errors, we cannot formulate $w$ simply as $w = X^{-1} y$.

### Exercise 1: Inverse Calculation
<b><div style="text-align:right">[POINTS: 5]</div></b>
Next, we try to obtain the case where inverse will be defined. For this, we multiply both sides of $X w = y$ by $X^T$ obtaining:

$$X^T X w = X ^ T y$$

<center>
$w = (X^T X)^{-1} X ^ T y$, provided $(X^T X)^{-1}$ exists
</center>

**Task:** 
- Computer $X^T X$


In [60]:
def get_X_transpose_X (X):
    X_T_X = None
    # YOUR CODE HERE
    X_T_X = np.transpose(X) @ X
    return X_T_X

X_transpose_X = get_X_transpose_X(X)

In [61]:
### INTENTIONALLY LEFT BLANK

In [62]:
# Get shape of X_transpose_X
X_transpose_X.shape

(10, 10)

Since, $X^T X$ is square matrix, it may be invertible unlike the rectangular matrix $X$. 

In [63]:
# Calculate inverse of X_transpose_X
try: 
    inv(X_transpose_X) 
except LinAlgError as err:
    print ("Error : " + str(err))

Error : Singular matrix is not invertible.


As the error says X_transpose_X is a singular matrix, so its inverse does not exist. Let's verify it.

In [64]:
print("Determinant of X_transpose_X = {}".format(det(X_transpose_X)))
print("Rank of X_transpose_X = {}".format(rank(X_transpose_X)))
print("Order of X_transpose_X = {}".format(X_transpose_X.shape[0]))

Determinant of X_transpose_X = 0.0
Rank of X_transpose_X = 8
Order of X_transpose_X = 10


Here `X_transpose_X` has rank 8 which is 2 less than its order. Hence, although it's a square matrix, its inverse does not exist. This infers that `X` has rank of 8 since `X_transpose_X` is matrix product of `X_transpose` and `X`. And, since rank of `X` is 8, there must be only 8 independent columns or only 8 independent rows or both. Since, the rows correspond to data for 1000 houses, it's highly unlikely that only 8 of the rows are independent unless it's a corrupted data. So, first let's check for the columns or the features of the data.       

It can be seen that `Circumference`, `Length` and `Width` may dependent. This dependency can be checked as follows: 

In [65]:
(df['Circumference'] == 2 * (df['Length'] + df['Width'])).all()

True

Thus, we have obtained one dependent column `Circumference` as it depends on `Length` and `Width`. There may be other independent variable as well. The two columns `Year_built` and `Age` may be dependent. The data was based on the selling of the houses in year 2019 and thus age was difference of 2019 and `Built_Age`. Since, we concatenated a vector of ones and data from columns of dataframe, data from columns `Year_built`, `Age` and 1<sup>st</sup> column in data matrix may be dependent. It can be checked as follows.

In [66]:
(df['Age'] == 2019 - df['Year_built']).all()

True

Now, we can remove one of $Circumference$ or $Length$ or $Width$ and one of $Year\_built$ or $Age$. Let's remove $Circumference$ and $Year\_built$ in the dataframe and compute $w$ again.

In [68]:
df_ = df.drop(columns=['Circumference', 'Year_built', 'Price'])
X = np.concatenate([np.ones((1000, 1)), df_.values], axis=1)
y = df['Price'].values

X_transpose_X = get_X_transpose_X(X)

print("Shape of X = {}".format(X.shape))
print("Rank of X = {}".format(rank(X)))

print()

print("Shape of X_transpose_X = {}".format(X_transpose_X.shape))
print("Order of X_transpose_X = {}".format(X_transpose_X.shape[0]))
print("Determinant of X_transpose_X = {}".format(det(X_transpose_X)))
print("Rank of X_transpose_X = {}".format(rank(X_transpose_X)))

Shape of X = (1000, 8)
Rank of X = 8

Shape of X_transpose_X = (8, 8)
Order of X_transpose_X = 8
Determinant of X_transpose_X = 1.4470596477559413e+40
Rank of X_transpose_X = 8


### Exercise 2: Calculate Left inverse of X
<b><div style="text-align:right">[POINTS: 5]</div></b>
Now we can calculate $w$ as $w = (X^T X)^{-1} X ^ T y$, since the $(X^T X)^{-1}$ exists after removing dependent features. But first we need to calcualte the left part of y, which is termed as left inverse of X whenever it exits.
<center>$X^{-1}_{left} = (X^TX)^{-1}X^T$</center>

**Task:** 
- Compute left inverse of $X$: $X^{-1}_{left}$


In [74]:
def left_inverse_of(X):
    lft_inv = None
    # YOUR CODE HERE
    lft_inv = inv(get_X_transpose_X(X)) @ X.T
    return lft_inv

left_inverse = left_inverse_of(X)

In [75]:
### INTENTIONALLY LEFT BLANK

Now, let's check if the left inverse we calculated is truely a left inverse. For this, check whether matrix product of left inverse of X and X give an identity matrix as follows. The matrix elements are rounded off by 7 since, the entire computation involves discripancies in numerical representation of computer hardware which become clear in numerical methods.  

In [76]:
(left_inverse @ X).round(7)

array([[ 1., -0., -0., -0., -0., -0., -0., -0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [-0., -0., -0.,  1., -0., -0., -0., -0.],
       [-0., -0., -0., -0.,  1., -0., -0., -0.],
       [-0., -0., -0., -0., -0.,  1., -0., -0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  0., -0.,  1.]])

In [77]:
((left_inverse @ X).round(7) == np.eye(rank(X))).all()

True

So, the left inverse matrix we calculated is correct.

### Exercise 3: 
<b><div style="text-align: right">[POINTS: 5]</div></b>

Now, obtain weight($w$) as we have calculated left inverse of $X$.

<center>
$w = (X^T X)^{-1} X ^ T y$, provided $(X^T X)^{-1}$ exists
</center>

**Task:** 
- compute weight w as $w = X^{-1}_{left}y$

In [85]:
def get_weights(X, y):
    wt = None
    # YOUR CODE HERE
    wt = left_inverse_of(X) @ y
    return wt

w = get_weights(X, y)

In [79]:
### INTENTIONALLY LEFT BLANK

Having obtained the weights vector $w$ from $w = (X^T X)^{-1} X^T y$, let's check if $X w = y$ is valid or not. 

In [82]:
(X @ w == y).any()

False

This has been the case since, $y$ is not contained in the column space of $X$. To do extra. If we consider the dataset ('housing_data.csv') is iid (independent and identically distributed), $X$ will have $1000$ (no of rows in $X$) independent rows but there are only $8$ independent columns. Hence, the column space will be a $8$ dimensional sub-space in $1000$  dimensional vector space, making the $Xw$ for any $w$ a very thin sub-space. Thus, the probability of $y$ being contained in the column space of $X$ will be very low when the real life data is taken. This will reason why the target variable price ($y$ in this case) is not perfectly output by $Xw$. There will be some error in that approximation and what $w = left\_inverse\_of(X) . y$ gives will be the minimum possible error for the case.